In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../src')
from feature_engineering import (
    load_and_preprocess_price_data,
    load_and_preprocess_scada_data,
    load_and_preprocess_fault_data,
    align_and_calculate_revenue,
)
from optimization_class import MaintenanceOptimization

# Simple approach. Only Faults and cost.

## Assumption: Only one maintenance type.
## Assumption 2: Each fault type only takes 1 trip per day

## **Optimization Logic**
### **Objective Function**
Maximize the **total profit**, which is:
\[
\text{Profit} = \text{Total Revenue} - (\text{Total Maintenance Cost} + \text{Lost Revenue})
\]
Where:
- **Total Revenue**: Revenue generated by turbines during operational hours.
- **Total Maintenance Cost**: Includes costs for internal, external, and preventative maintenance.
- **Lost Revenue**: Revenue lost due to turbines being non-operational during fault periods.


### **Variables**
1. **Internal Maintenance (`Internal_F`)**:
   - Fixed annual cost (\$750,000).
   - Address faults without requiring external intervention.
2. **External Maintenance (`External_F_D`)**:
   - Costs vary by season:
     - High-demand months: \$150,000.
     - Normal-demand months: \$50,000.
   - Can address faults on specific days.
3. **Preventative Maintenance (`Preventative_F`)**:
   - Fixed cost (\$50,000).
   - Reduces fault probability but does not directly repair faults.

### **Constraints**
1. **At Least One Maintenance Method Per Fault**:
   For each fault `F` on each day `D`:
   \[
   \text{Internal}_F + \text{External}_{F,D} \geq 1
   \]
   Ensures all faults are addressed.

2. **Exclusive Maintenance Selection**:
   Internal and external maintenance cannot be selected simultaneously:
   \[
   \text{Internal}_F + \text{External}_{F,D} \leq 1
   \]

3. **Preventative Maintenance**:
   Preventative maintenance is optional:
   \[
   \text{Preventative}_F \leq 1
   \]

### **Revenue Loss Calculation**
If a fault occurs and no maintenance is performed, the turbine is non-operational, leading to:
\[
\text{Lost Revenue} = \text{Production (kWh)} \times \text{Price (\$/MWh)} / 1000
\]
This value is added to the total cost during optimization.

# Optimization Implementation:

In [3]:
# Paths to the datasets
price_data_path = '../data/raw/rpt.00013060.0000000000000000.DAMLZHBSPP_2014.xlsx'
scada_data_path = '../data/raw/scada_data.csv'
fault_data_path = '../data/raw/fault_data.csv'

# Load and preprocess data
price_data_sheets = ['May_5', 'Jun_6', 'Jul_7', 'Aug_8', 'Sep_9', 'Oct_10', 'Nov_11', 'Dec_12']
price_data_houston = load_and_preprocess_price_data(price_data_path, price_data_sheets)
scada_data_processed = load_and_preprocess_scada_data(scada_data_path, price_data_houston['DateTime'].min(), price_data_houston['DateTime'].max())
fault_data = load_and_preprocess_fault_data(fault_data_path)

# Extract fault types and days
fault_types = fault_data['Fault'].unique()
days = fault_data['Day'].unique()

# Instantiate and solve optimization
optimizer = MaintenanceOptimization(data=fault_data, revenue_data=merged_data)
optimizer.initialize_variables(fault_types, days)
optimizer.define_problem(fault_types, days)
optimizer.solve()

# Retrieve and display results
results = optimizer.get_results()
print("Optimized Results:")
print(f" - Internal Maintenance Cost: ${results['optimized_internal_cost']:.2f}")
print(f" - External Maintenance Cost: ${results['optimized_external_cost']:.2f}")
print(f" - Preventative Maintenance Cost: ${results['optimized_preventative_cost']:.2f}")
print(f" - Total Maintenance Cost: ${results['total_cost']:.2f}")
print(f" - Total Revenue: ${results['total_revenue']:.2f}")

Optimized Results:
 - Internal Maintenance Cost: $3750000.00
 - External Maintenance Cost: $0.00
 - Preventative Maintenance Cost: $0.00
 - Total Maintenance Cost: $3750000.00
 - Total Revenue: $757923247.03


## Final Solution
Use internal maintenance